In [ ]:
from misc_utils.train_utils import (
    unit_test_create_dataset,
    unit_test_create_diffusion_model,
    unit_test_diffusion_val_step,
    unit_test_diffusion_training_step
)
config_file = 'configs/mean_flow_mnist.yaml'

In [ ]:
res = unit_test_create_dataset(config_file)

In [ ]:
res = unit_test_create_diffusion_model(config_file)

In [ ]:
res = unit_test_diffusion_val_step(config_file)

In [ ]:
res = unit_test_diffusion_training_step(config_file)

In [ ]:
from dataloaders.mnist import MNISTDataset
dataset = MNISTDataset(root='.', download=True, train=True)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(dataset[0]['image'])


In [ ]:
from models.mean_flow_model import DualTimestepDiTTransformer2DModel

model =DualTimestepDiTTransformer2DModel(
    in_channels=1,
    out_channels=1,
    num_layers=6,
    num_attention_heads=8,
    attention_head_dim=16,
)

In [ ]:
from diffusers.models.attention import Attention
from diffusers.models.attention_processor import AttnProcessor

for name, m in model.named_modules():
    if isinstance(m, Attention):
        m.set_processor(AttnProcessor())
        print(m.processor.__class__)

In [ ]:
from pipelines.mean_flow_pipeline import MeanFlowPipeline
pipe = MeanFlowPipeline(
    denoiser=model
)

In [ ]:
sample = pipe(
    height=28, width=28,
    num_samples=4,
    output_type='np'
)

In [ ]:
sample[0].shape

In [ ]:
import torch
x = torch.randn(1, 3, 32, 32)
t = torch.tensor([0.5])
r = torch.tensor([0.2])
cls_label =  torch.tensor([1], dtype=torch.int64)
o1 = model(x, t, r, class_labels=cls_label)
o2 = model(x, t, r+0.1, class_labels=cls_label)
o3 = model(x, t+0.1, r, class_labels=cls_label)

In [ ]:
print((o1.sample - o2.sample).abs().sum())
print((o1.sample - o3.sample).abs().sum())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import lognorm

mu = -2
sigma = 2

x = np.linspace(1e-5, 1, 1000)
pdf = lognorm.pdf(x, s=sigma, scale=np.exp(mu))

plt.plot(x, pdf)
plt.title('Log-normal PDF (mu=-2, sigma=2)')
plt.xlabel('x')
plt.ylabel('Density')
plt.grid(True)
plt.show()


In [ ]:
import torch
x = torch.distributions.log_normal.LogNormal(loc=torch.tensor([-2.0]), scale=torch.tensor([2.0]))
x.sample((100,))